In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

# 1.逻辑回归公式小结

## 1.1 符号说明

<font size=4>
$x^{(i)}: 输入变量，是一个n×1的列向量$  
$y^{(i)}: 输出变量/目标变量，是一个实数，取值为0或者1$  
$\theta: 参数，是一个n×1的向量$  
$(x^{(i)}, y^{(i)}): 一个训练样本$  
$\{(x^{(i)}, y^{(i)}); i = 1,...,m\}:训练集$  
$m: 训练样本的个数$  
$n: 输入变量的特征数$
$X: 是一个m×n的矩阵。其中，每一行是输入变量的转置，每一列对应每一个特征。$  
$$X=\begin{bmatrix} - (x^{(1)})^T -\\ - (x^{(2)})^T -\\...\\ - (x^{(m)})^T -\end{bmatrix}\quad$$
$\vec{y}:是一个m×1的列向量，其中，每一行对应每一个输入变量的真实标签。$  
$$\vec{y}=\begin{bmatrix} y^{(1)} \\ y^{(2)} \\ ... \\ y^{(m)} \end{bmatrix}\quad$$
</font>

## 1.2 主要公式

### 1.2.1 Cost Function

<font size=4>
$$J(\theta)=\frac{1}{m}\sum_{i=1}^{m}[-y^{(i)}log(h_{\theta}(x^{(i)}))-(1-y^{(i)})log(1-h_{\theta}(x^{(i)}))]$$
</font>

<font size=4>
&emsp;&emsp;使用 $\circ$ 表示向量对应位置元素相乘，即 element-wise multiplication 或者 hadamard product。则将cost function可用向量表示为：
</font>

<font size=4>
&emsp;&emsp;$$J(\theta)=\frac{1}{m}[(-\vec{y})\,{\circ}\,log(h_{\theta}(X))-(1-\vec{y})\,{\circ}\,log(1-h_{\theta}(X))]$$
</font>

### 1.2.2 Gradient

<font size=4>
&emsp;&emsp;对 $\theta$ 的第 j 个分量求梯度可表示为：
</font>

<font size=4>
$$\frac{{\partial}J(\theta)}{\partial{\theta}_j}=\frac{1}{m}\sum^{m}_{i=1}(h_{\theta}(x^{(i)})-y^{(i)})x^{(i)}_j$$
</font>

<font size=4>
&emsp;&emsp;用向量进行表示为：
</font>

<font size=4>
$$\frac{\partial{J(\theta)}}{\partial{\theta}}=\frac{1}{m}X^T(h_{\theta}(X)-\vec{y})$$
</font>

<font size=4>
&emsp;&emsp;注意到 $h_{\theta}(X)-\vec{y}$ 是一个 m×1 的向量，即
</font>

<font size=4>
$$h_{\theta}(X)-\vec{y}=\begin{bmatrix} h_{\theta}(x^{(1)})-y^{(1)} \\ h_{\theta}(x^{(2)})-y^{(2)} \\ ... \\ h_{\theta}(x^{(m)})-y^{(m)}  \end{bmatrix}\quad$$
</font>

### 1.2.3 Hessian Matrix

<font size=4>
&emsp;&emsp;根据上面梯度的公式，我们可以计算出 Hessian Matrix 的第 i 行，第 j 列分量为：
</font>

<font size=4>
\begin{align}
H_{ij} & = \frac{\partial^2J(\theta)}{\partial{\theta}_i{\theta}_j} \\
& = \frac{1}{m}\frac{\partial}{\theta_j}\sum^{m}_{k=1}(h_{\theta}(x^{(k)})-y^{(k)})x^{(k)}_i \\
& = \frac{1}{m}\sum^{m}_{k=1}\frac{\partial}{\theta_j}(h_{\theta}(x^{(k)})-y^{(k)})x^{(k)}_i \\
& = \frac{1}{m}\sum^{m}_{k=1}x^{(k)}_i\frac{\partial}{\theta_j}h_{\theta}(x^{(k)}) \\
& = \frac{1}{m}\sum^{m}_{k=1}x^{(k)}_ih_{\theta}(x^{(k)})(1-h_{\theta}(x^{(k)}))\frac{\partial}{\theta_j}(\theta^Tx^{(k)}) \\
& = \frac{1}{m}\sum^{m}_{k=1}x^{(k)}_ih_{\theta}(x^{(k)})(1-h_{\theta}(x^{(k)}))x^{(k)}_j
\end{align}
</font>

<font size=4>
&emsp;&emsp;因此，Hessian Matrix可表示为：
</font>

<font size=4>
\begin{align*}
H & =\frac{1}{m}\sum^{m}_{k=1}h_{\theta}(x^{(k)})(1-h_{\theta}(x^{(k)}))x^{(k)}(x^{(k)})^T \\
& = \frac{1}{m}X^TWX
\end{align*}
</font>

<font size=4>
&emsp;&emsp;其中，矩阵W是一个 m×m 的对角矩阵，主对角线上的元素为 $h_{\theta}(x^{(k)})(1-h_{\theta}(x^{(k)}))$ (k=1,...,m)
</font>

# 2.牛顿法原理

<font size=4>
&emsp;&emsp;本节主要参考 **peghoty** 对于牛顿法的介绍，网址为http://blog.csdn.net/itplus/article/details/21896453
</font>

# 3.使用牛顿法求解逻辑回归参数

In [ ]:
<font size=4>
&emsp;&emsp;
</font>